SCRIPT SCRAPING DI ODDS PORTAL
================

### Descrizione generale dello script
*Per far funzionare lo script è necessario modificare il percorso del file chromedriver.exe, 
*o inserire nella cartella il proprio file.
* La classe Scraper prende gli url dalla cartella leagues e crea un database dove inserisce i risultati
* Estrae i dati dai tag html rilevanti
* Aggiorna le righe con le informazioni raccolte

![script](figura1.png)

### Dati di interesse
*Al fine di costruire la (futura) rete neurale, abbiamo selezionato le informazioni per noi rilevanti 
* Abbiamo costruito lo scraper in modo tale che salvesse le seguenti informazioni:
    * I nomi delle 2 squadre del singolo match
    * L'inizio e la fine delle quote 
    * Il risultato finale della partita
    * Il campionato di appartenenza e l'area
    * Il sito da cui prevenivano le singole quote
    * La quota di 1,X,2
* In questo modo per ogni partita abbiamo più righe, ogni riga dedicata all'estrazione delle informazioni da una fonte diversa
* L'obiettivo di questo Scraper è quello di creare un dataset il più completo possibile, di dimensioni elevate

![script](figura2.png)

In [43]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [10]:
df = pd.read_csv('./matches.csv')

In [11]:
df.head()

,league,area,retrieved_from_url,start_time,end_time,team1,team2,outcome,team1_odds,team2_odds,draw_odds
0,Europa League,Europe,http://www.oddsportal.com/soccer/europe/europa...,1530204300,1530209700,Tre Fiori (San),Bala (Wal),TEAM1,7.50,1.33,4.78
1,Europa League,Europe,http://www.oddsportal.com/soccer/europe/europa...,1530201600,1530207000,B36 Torshavn (Fai),St Josephs (Gib),DRAW,1.34,6.80,4.90
2,Europa League,Europe,http://www.oddsportal.com/soccer/europe/europa...,1530201600,1530207000,Druids (Wal),Trakai (Ltu),DRAW,5.50,1.54,3.83
3,Europa League,Europe,http://www.oddsportal.com/soccer/europe/europa...,1530201600,1530207000,UE Engordany (And),Folgore (San),TEAM1,1.44,7.27,3.75
4,Europa League,Europe,http://www.oddsportal.com/soccer/europe/europa...,1530198000,1530203400,Birkirkara (Mlt),Klaksvik (Fai),DRAW,1.59,5.32,3.48


In [94]:
len(df) # dimensioni dataset

457

### OneHotEncoding sulla colonna 'outcome'

In [17]:
outcome_encoded = pd.get_dummies(df['outcome']) # faccio OneHotEncoding sulla colonna 'outcome'
df = pd.concat([df, outcome_encoded], axis = 1) # concateno le colonne risultato del OHE a df

In [22]:
df.head(1)

,league,area,retrieved_from_url,start_time,end_time,team1,team2,outcome,team1_odds,team2_odds,draw_odds,DRAW,TEAM1,TEAM2
0,Europa League,Europe,http://www.oddsportal.com/soccer/europe/europa...,1530204300,1530209700,Tre Fiori (San),Bala (Wal),TEAM1,7.5,1.33,4.78,0,1,0


### Features
* Quota 1
* Quota 2
* Quota X

In [19]:
X = df.loc[:, ['team1_odds', 'team2_odds', 'draw_odds']].values # values per convertire il DataFrame in numpy array

In [20]:
X

array([[7.5 , 1.33, 4.78],
       [1.34, 6.8 , 4.9 ],
       [5.5 , 1.54, 3.83],
       ...,
       [4.75, 1.65, 3.5 ],
       [3.3 , 2.04, 3.2 ],
       [4.  , 1.85, 3.2 ]])

In [21]:
X.shape # 457 righe, 3 colonne

(457, 3)

### Colonna Target
* Vogliamo predire l'esito (DRAW, TEAM1, TEAM2)

In [25]:
y = df.loc[:, ['DRAW', 'TEAM1', 'TEAM2']].values

In [26]:
y

array([[0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [27]:
y.shape

(457, 3)

### Split training-test set

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [34]:
X_train.shape

(365, 3)

In [35]:
y_train.shape

(365, 3)

### Classificazione
* Date le quote dell'1, dell'X e del 2 voglio predire l'esito della partita
* L'esito della partita è una classe
* In totale ho 3 classi (DRAW, TEAM1, TEAM2)
    * Un vettore (1, 0, 0) -> DRAW
    * Un vettore (0, 1, 0) -> TEAM1
    * Un vettore (0, 0, 1) -> TEAM2
* Questo è un problema di **multiclass classification**
    * In sklearn gli unici classificatori che implementano una classificazione multiclasse sono
        * Decision trees
        * Random Forest
        * Nearest Neighbours
    * In via *"eccezionale"* è possibile usare anche la LogisticRegression
        * OneVsRestClassification
        * Ma è una forzatura
* Ho deciso di utilizzare Random Forest

In [87]:
# n_estimators = numero di alberi, si può giocare un po' con questo valore
classifier = RandomForestClassifier(n_estimators=10, criterion='entropy')

In [88]:
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [91]:
y_predicted_test = classifier.predict(X_train)
y_predicted = classifier.predict(X_test)

In [92]:
accuracy_score(y_train, y_predicted_test, normalize=True, sample_weight=None)

0.8575342465753425

In [93]:
accuracy_score(y_test, y_predicted, normalize=True, sample_weight=None)

0.3695652173913043

### Problemi
* Accuracy bassissima sul test set ed abbastanza alta sul training set
    * Chiaro segno di **overfitting**
* Perché?
    * Troppe poche features considerate (solo quote 1, X, 2)
        * Troppo difficile generalizzare pattern solo su questi 3 valori
    * Troppe poche righe nel dataset (457, da splittare tra training e test set)
        * Ne servono **almeno** 10 volte di più

### Nuovo Dataset
* Ampliamo il nostro dataset utilizzando lo script, raccogliendo anche informazioni circa altri campionati/leghe
    * Ripetiamo i precedenti passi
* Che risultati ci aspettiamo?
    * Sicuramente un aumento, seppur lieve, della accuracy
        * Con il nuovo dataset di più di 3000 elementi possiamo suddividere meglio training set e test set

In [12]:
df = pd.read_csv('./matches_new.csv')

In [13]:
len(df) # dimensioni dataset

3196

In [14]:
#Ripeto i passaggi precedenti con il nuovo dataset
outcome_encoded = pd.get_dummies(df['outcome']) # faccio OneHotEncoding sulla colonna 'outcome'
df = pd.concat([df, outcome_encoded], axis = 1) # concateno le colonne risultato del OHE a df
X = df.loc[:, ['team1_odds', 'team2_odds', 'draw_odds']].values # values per convertire il DataFrame in numpy array
y = df.loc[:, ['DRAW', 'TEAM1', 'TEAM2']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
classifier = RandomForestClassifier(n_estimators=10, criterion='entropy')
classifier.fit(X_train, y_train)
y_predicted_test = classifier.predict(X_train)
y_predicted = classifier.predict(X_test)

In [15]:
accuracy_score(y_train, y_predicted_test, normalize=True, sample_weight=None)

0.9107981220657277

In [23]:
accuracy_score(y_test, y_predicted, normalize=True, sample_weight=None)

0.603125

In [29]:
pred_set = []
for index, row in df.iterrows():
    if row['team1_odds'] < row['team2_odds']:
        pred_set.append({'team1': row['team1'], 'team2': row['team2'], 'winning_team': None})
    else:
        pred_set.append({'team1': row['team1'], 'team2': row['team2'], 'winning_team': None})
pred_set = pd.DataFrame(pred_set)

In [30]:
backup_pred_set = pred_set

pred_set.head()

,team1,team2,winning_team
0,Tre Fiori (San),Bala (Wal),None
1,B36 Torshavn (Fai),St Josephs (Gib),None
2,Druids (Wal),Trakai (Ltu),None
3,UE Engordany (And),Folgore (San),None
4,Birkirkara (Mlt),Klaksvik (Fai),None


In [41]:
# Add missing columns compared to the model's training dataset
missing_cols = set(outcome_encoded.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[outcome_encoded.columns]

# Remove winning team column


pred_set.head()

,DRAW,TEAM1,TEAM2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
